In [1]:
from pathlib import Path
from tqdm import tqdm

from matplotlib import pyplot as plt
#import mne
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import accuracy_score
import statsmodels.api as sm

In [2]:
# Create the colormap
colors = [(0, 191/255, 149/255), (191/255,0,42/255)]
ur2 = sns.color_palette(colors)

Load trial data

In [3]:
path_data = Path('trials_BriefAC-v2.csv')
df_trials = pd.read_csv(path_data)

# Replace keyYes with it's keyboard counterpart
def convertKey(x):
    if x == 'left':
        x = 37
    elif x == 'right':
        x = 39
    return x

def context2scene(x):
    if x == 'context':
        x = 'Scene'
    elif x == 'action':
        x = 'Action'
    return x

df_trials.keyYes = df_trials.keyYes.apply(convertKey)

# Replace empty responses to 0-s
df_trials.Response = df_trials.Response.apply(lambda x: 0 if pd.isna(x) else x)

# Replace empty responses to 0-s
df_trials.ProbeType = df_trials.ProbeType.apply(context2scene)

# Round PT to integer
df_trials = df_trials.round({'PT' : 0, 'RT': 1})

# Convert 'PT' and 'Response' columns to integers
df_trials = df_trials.astype({'PT': 'int', 'Response': 'int'})

df_trials.head()

,subID,keyYes,trialNr,picID,Context,Action,Compatibility,PT,ProbeType,Probe,CorrectResponse,Response,RT
0,1,39,3,389,workshop,hammering,compatible,83,Scene,workshop,39,37,1221.3
1,1,39,4,315,office,hammering,incompatible,50,Action,hammering,39,39,1014.3
2,1,39,5,324,office,sawing,incompatible,50,Action,stamping,37,37,921.3
3,1,39,6,341,office,hole-punching,compatible,33,Scene,workshop,37,37,688.0
4,1,39,7,267,kitchen,hole-punching,incompatible,50,Action,hole-punching,39,39,690.1


Extract response statistics (hits, misses, false alarms and correct rejections)
* **Hit** : Yes when "yes" is correct
* **False Alarm** : Yes when "no" is correct
* **Correct Rejection** : No when "no" is correct
* **Miss** : No when "yes" is correct

Factors of interest:
* PT
* Compatibility
* ProbeType

Check in dataframe whether each participants has all:

* PTs
* Compatibilities
*

In [4]:
def summary(df):
    """Given a set of trials, extract summary data:
    * Hit : Yes when "yes" is correct
    * False Alarm : Yes when "no" is correct
    * Correct Rejection : No when "no" is correct
    * Miss : No when "yes" is correct
    """
    n_hits = 0;
    n_misses = 0;
    n_correjections = 0;
    n_falarms = 0;
    n_empty = 0;
    
    # Iterate over trials (rows of subset)
    for index, row in df.iterrows():
        # Ignore empty
        if row.Response == 0:
            n_empty += 1
            continue
        
        # YES-responses
        if row.Response == row.keyYes:
            if row.Response == row.CorrectResponse:
                n_hits += 1
            else:
                n_falarms += 1
        # NO-responses
        else:
            if row.Response == row.CorrectResponse:
                n_correjections += 1
            else:
                n_misses += 1
    
    #print(n_empty)
    
    return (n_hits, n_falarms, n_correjections, n_misses)

In [5]:
from scipy.stats import norm

def dprime(n_hits, n_falarms, n_correjections, n_misses):
    """Computes sensitivity (d-prime index) as defined in MacMillan & Creelman (2005)"""
    # Compute H- & F-rates
    if n_hits == 0: # H = 0
        H = 1 / (2*(n_hits + n_misses))
    elif n_misses == 0: # H = 1
        H = 1 - 1 / (2*(n_hits + n_misses))
    else:
        H = n_hits / (n_hits + n_misses)
        
    if n_falarms == 0: # F = 0
        F = 1 / (2*(n_falarms + n_correjections))
    elif n_correjections == 0: # F = 1
        F = 1 - 1 / (2*(n_falarms + n_correjections))
    else:
        F = n_falarms / (n_falarms + n_correjections)
    
    d = norm.ppf(H) - norm.ppf(F)
    
    # Compute perfect score (max d-prime) w/ assumptions:
    # = Hits: 1 - 1/(2N) = 1 - 1/[2*(Hits + Misses)];
    # = FalseAlarms: 1/2N = 1/2*(FalseAlarms + CorrectRejections)
    H_ideal = 1 - 1 / (2*(n_hits + n_misses))
    F_ideal = 1 / (2*(n_falarms + n_correjections))
    d_ideal = norm.ppf(H_ideal) - norm.ppf(F_ideal)
    
    return d

In [6]:
def accuracy(y_pred, y_true):
    """Computes sensitivity (d-prime index) as defined in MacMillan & Creelman (2005)"""
    #score = y_pred = subset.Response
    #y_true = subset.CorrectResponse
    return accuracy_score(y_true, y_pred)

In [7]:
a_subs = np.unique(df_trials.subID)
a_pts  = np.unique(df_trials.PT)
a_comp = np.unique(df_trials.Compatibility)
a_prob = np.unique(df_trials.ProbeType)

l_data = []

# iterate over subjects ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
for i in range(len(a_subs)):
    #print(i)
    n_samples = 0;
    n_empty = 0;
    hits = 0;
    misses = 0;
    corr_rejections = 0;
    f_alarms = 0;

    # iterate over PTs ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    for j in range(len(a_pts)):
        #print(a_subs[i], a_pts[j])
        # iterate over Compatibility ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        for k in range(len(a_comp)):
            #print(a_subs[i], a_pts[j], a_comp[k])
            # iterate over ProbeType ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            for l in range(len(a_prob)):
                subset = df_trials[
                            (df_trials.subID == a_subs[i]) &
                            (df_trials.PT == a_pts[j]) &
                            (df_trials.Compatibility == a_comp[k]) &
                            (df_trials.ProbeType == a_prob[l])]
                (n_hits, n_falarms, n_correjections, n_misses) = summary(subset)
                d = dprime(n_hits, n_falarms, n_correjections, n_misses)
                
                l_data.append([a_subs[i], a_pts[j], a_comp[k], a_prob[l],
                               n_hits, n_falarms, n_correjections, n_misses, d])
                #print(a_subs[i], a_pts[j], a_comp[k], a_prob[l], subset.shape)

In [8]:
df_summary = pd.DataFrame(l_data,
                          columns=['subID', 'PT', 'Compatibility', 'ProbeType',
                                   'n_hits', 'n_falarms', 'n_correjections',
                                   'n_misses', 'dprime'])

#df_summary.to_csv('summary_data.csv')

In [9]:
df_summary

,subID,PT,Compatibility,ProbeType,n_hits,n_falarms,n_correjections,n_misses,dprime
0,1,33,compatible,Action,2,0,18,16,0.693865
1,1,33,compatible,Scene,5,3,15,13,0.377966
2,1,33,incompatible,Action,3,1,17,14,0.664319
3,1,33,incompatible,Scene,2,3,15,16,-0.253219
4,1,50,compatible,Action,4,7,11,14,-0.482494
...,...,...,...,...,...,...,...,...,...
211,9,117,incompatible,Scene,0,1,17,18,-0.321287
212,9,150,compatible,Action,17,11,7,1,1.311003
213,9,150,compatible,Scene,0,0,18,18,0.000000
214,9,150,incompatible,Action,17,8,10,1,1.732929


# Permutation cluster test

## Actions : Compatible vs Incompatible

In [10]:
# Actions compatible
condition1 = df_summary[(df_summary.ProbeType == 'Action') &
                        (df_summary.Compatibility == 'compatible')][
                            ["dprime", "PT"]].sort_values(["PT", "dprime"])
condition2 = df_summary[(df_summary.ProbeType == 'Action') &
                         (df_summary.Compatibility == 'incompatible')][
                             ["dprime", "PT"]].sort_values(["PT", "dprime"])
                         
condition1.head()

,dprime,PT
144,-0.282216,33
96,-0.139710,33
192,0.372578,33
168,0.377966,33
48,0.589456,33


#### 1) Compare conditions by means of t-tests

In [11]:
import scipy
from scipy.stats import ttest_ind, f

t_stats = []
for pt in np.unique(condition1.PT):
    results = ttest_ind(condition1.dprime[condition1.PT == pt],
                              condition2.dprime[condition2.PT == pt])
    t_stats.append([pt, results.statistic, results.pvalue])
    
t_stats = np.array(t_stats)
print(t_stats)

[[ 3.30000000e+01  1.00185679e+00  3.31324576e-01]
 [ 5.00000000e+01  9.42284971e-01  3.60057965e-01]
 [ 6.70000000e+01  5.14365146e-01  6.14028086e-01]
 [ 8.30000000e+01 -1.31471611e+00  2.07139318e-01]
 [ 1.17000000e+02 -1.20170673e+00  2.46962971e-01]
 [ 1.50000000e+02  5.84201061e-02  9.54137412e-01]]


#### 2) Given a threshold, select t-values higher than it

In [12]:
pval = 0.025  # arbitrary
n_conditions = 2
n_observations = 9
df = n_observations - n_conditions  # degrees of freedom
thresh = scipy.stats.t.ppf(1 - pval, df)  # T distribution
print(thresh)

2.3646242510102993


## MNE try

In [63]:
from mne.stats import permutation_cluster_test


threshold = 0.2
T_obs, clusters, cluster_p_values, H0 = \
    permutation_cluster_test([condition1, condition2],
                             n_permutations=1000,
                             threshold=threshold, tail=1, n_jobs=None,
                             out_type='indices',
                             adjacency=None)

stat_fun(H1): min=33.314483 max=33.314483
Running initial clustering …
Found 1 cluster


100%|██████████| Permuting : 999/999 [00:00<00:00, 8989.74it/s]


In [64]:
clusters

[(array([0]),)]

In [42]:
# Actions compatible
condition1 = df_summary.dprime[(df_summary.ProbeType == 'Action') &
                               (df_summary.Compatibility == 'compatible')]
condition2 = df_summary.dprime[(df_summary.ProbeType == 'Action') &
                               (df_summary.Compatibility == 'incompatible')]

threshold = 0.0
T_obs, clusters, cluster_p_values, H0 = \
    permutation_cluster_test([condition1, condition2],
                             n_permutations=1000,
                             threshold=threshold, tail=1, n_jobs=None,
                             out_type='mask',
                             adjacency=None)

stat_fun(H1): min=0.000607 max=0.000607
Running initial clustering …
Found 1 cluster


/home/ov/anaconda3/envs/mne/lib/python3.9/site-packages/mne/stats/cluster_level.py:852: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  X = [x[:, np.newaxis] if x.ndim == 1 else x for x in X]
100%|██████████| Permuting : 999/999 [00:00<00:00, 6829.93it/s]
